# Transactions, Validation, and Updating system state
A blockchain is a distributed database with a set of rules for verifying new additions to the database.
We are taking an example account of `Tharun` and `Sam` who will trade virtual money with each other.

## What we are doing?
We should to create a transaction pool of incoming transactions, validate those transactions, and make them into a block.

## How are we doing?
We will use a hash function (to make it easy we will use the `hashlib` module) to create a unique hash for each of the transactions- this hash function links each of the blocks to each other creating a block chain.

In [ ]:
# Importing required libs
import hashlib, json, sys
# creating a function with argument message
# Input: some message
def hashMe(msg=""):
    if type(msg)!=str:
        msg = json.dumps(msg,sort_keys=True)
        
    if sys.version_info.major == 2:
        return unicode(hashlib.sha256(msg).hexdigest(),'utf-8')
    else:
        return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()

# calling the function hashMe that encrypts the string into a 256 hash
# Just for example:
hashMe("Hello")

creating a function to generate exchanges between `Tharun` and `Sam`. Negative numbers are withdrawals and positive numbers will be deposits. We are constructing our transactions to always be between two users, and make sure that the deposit is the same magnitude as the withdrawal- i.e. that we’re neither creating nor destroying money. (😆 physics)

In [6]:
import random
random.seed(0)

def makeTransaction(maxValue=3): # this creates a transactions between 1 to 3
    sign      = int(random.getrandbits(1))*2 - 1   # randomly chooses - or +
    amount    = random.randint(1,maxValue) # creates random transaction amount btw 1 to 3
    tharunPays = sign * amount
    samPays   = -1 * tharunPays
    # This will always return transactions to respect conservation of tokens.
    return {u'Tharun':tharunPays,u'Sam':samPays}

# creating a random sets of transactions and adding them to the block
#txn stand for transaction
txnBuffer = [makeTransaction() for i in range(30)]

Now we are creating block for each of the transactions happened in tnxBuffer.
But before that we also need to verify the transactions that we are adding to the block.

In `bitcoin` it uses `proof of work & proof of state` which basically checks whether the after transactions amount is greater than before transactions amount and that the keys used for the signatures are valid.

But in `ethereum` its different it uses `smart contracts` which checks wether the contact is faithfully executed respect to the gas limits.

But in this we will create our own rules for verification since those are complex and complicated.

## Our rules for validating before adding to block
- The sum of deposits and withdrawals must be 0 (tokens can neither be created nor destroyed)
- A user’s account must have some minimum tokens to cover any withdrawals (so that he cant give token when he doesn't have token)

If either of these conditions are violated, it will reject the transaction.

In [7]:
def updateState(txn, state):
    # Inputs: txn, state: dictionaries keyed with account names, holding numeric values for transfer amount (txn) or account balance (state)
    # Outputs: Updated state, with additional users added to state if necessary
    # This doesn't need to validate the transaction cuz its just updating the state!
    
    # If the transaction is valid, then update the state
    state = state.copy() 
    # As dictionaries (dict are used to store data values in key:value pairs like json) are mutable,
    # So let's avoid any confusion by creating a working copy of the data.
    for key in txn:
        if key in state.keys():
            state[key] += txn[key]
        else:
            state[key] = txn[key]
    return state

In [ ]:
def isValidTxn(txn,state):
    # Assume that the transaction is a dictionary keyed by account names

    # Check that the sum of the deposits and withdrawals is 0
    if sum(txn.values()) is not 0:
        return False
    
    # Check that the transaction does not cause the account to overdrawn
    # what is overdrawn or overdraft:
    # a situation where u have 1000rs only in ur bank but u give 1500rs check to someone. This may cause the someone to 
    # pay penalty money known as overdraft (and the account u have is known as overdrawn account) 
    for key in txn.keys():
        if key in state.keys(): 
            acctBalance = state[key]
        else:
            acctBalance = 0
        if (acctBalance + txn[key]) < 0:
            return False
    
    return True

Time to check wether our validation works:
We are giving a set of transaction that dosen't fit our rules.

In [ ]:
state = {u'Tharun':5,u'Sam':5}

print(isValidTxn({u'Tharun': -3, u'Sam': 3},state))  # Basic transaction- this works great!
print(isValidTxn({u'Tharun': -4, u'Sam': 3},state))  # But we can't create or destroy tokens!
print(isValidTxn({u'Tharun': -6, u'Sam': 6},state))  # We also can't overdraft our account.
print(isValidTxn({u'Tharun': -4, u'Sam': 2,'Rahul':2},state)) # Creating new users is valid
print(isValidTxn({u'Tharun': -4, u'Sam': 3,'Rahul':2},state)) # But the same rules still apply!

## Building Blocks
Creating a block and adding the valid transactions.
At the start we have nothing on the block chain. But we can get things started by defining the ‘genesis block’ or 'parent block' or 'first block' of the system.
Since the genesis block isn’t linked to any prior block, it gets treated a bit differently, and we can arbitrarily set the system state. In our case, we’ll create accounts for our two users (`Tharun` and `Sam`) and give them 50 coins each.

In [ ]:
state = {u'Tharun':50, u'Sam':50}  # Defining the initial state
genesisBlockTxns = [state]
genesisBlockContents = {u'blockNumber':0,u'parentHash':None,u'txnCount':1,u'txns':genesisBlockTxns}
genesisHash = hashMe( genesisBlockContents )
genesisBlock = {u'hash':genesisHash,u'contents':genesisBlockContents}
genesisBlockStr = json.dumps(genesisBlock, sort_keys=True)
# For each block, we want to collect a set of transactions, create a header, hash it, and add it to the chain
chain = [genesisBlock]

In [ ]:
# Making block
def makeBlock(txns,chain):
    parentBlock = chain[-1]
    parentHash  = parentBlock[u'hash']
    blockNumber = parentBlock[u'contents'][u'blockNumber'] + 1
    txnCount    = len(txns)
    blockContents = {u'blockNumber':blockNumber,u'parentHash':parentHash,
                    u'txnCount':len(txns),'txns':txns}
    blockHash = hashMe( blockContents )
    block = {u'hash':blockHash,u'contents':blockContents}
    
    return block

In [ ]:
blockSizeLimit = 5  # Size limit of block transactions/data that a block can hold
#  this is chosen by the block miner, and can vary between blocks!

while len(txnBuffer) > 0:
    bufferStartSize = len(txnBuffer)
    
    ## Gather a set of valid transactions for inclusion
    txnList = []
    while (len(txnBuffer) > 0) & (len(txnList) < blockSizeLimit):
        newTxn = txnBuffer.pop()
        validTxn = isValidTxn(newTxn,state) # This will return False if txn is invalid
        
        if validTxn:           # If we got a valid state
            txnList.append(newTxn)
            state = updateState(newTxn,state)
        else:
            print("ignored transaction")
            sys.stdout.flush()
            continue  # If invalid transaction; ignore it and move on
        
    ## Make a block
    myBlock = makeBlock(txnList,chain)
    chain.append(myBlock)

Creating initial block and transaction

In [ ]:
chain[0]

Second block and transaction

chain[1]

As expected, the genesis block includes an invalid transaction which initiates account balances (creating tokens out of thin air). The hash of the parent block is referenced in the child block, which contains a set of new transactions which affect system state. We can now see the state of the system, updated to include the transactions:

In [ ]:
# to check the current state after transactions
state

## Checking chain validation (Linked blocks validation)
Now that we have created a block and linked them in a chain. we need to ensure and verify the chain so that no new block can be added in between and also to check new blocks are valid.

On a blockchain network, this becomes important in two ways:

- When we initially set up our node, we will download the full blockchain history. After downloading the chain, we would need to run through the blockchain to compute the state of the system. To protect against somebody inserting invalid transactions in the initial chain, we need to check the validity of the entire chain in this initial download.
- Once our node is synced with the network (has an up-to-date copy of the blockchain and a representation of system state) it will need to check the validity of new blocks that are broadcast to the network.

### We will need three functions to make this:
+ `checkBlockHash`: a simple function to check the content of the block matches with the hash of the block.
+ `checkBlockValidity`: Checks the validity of a block, given its parent and the current system state.
+ `checkChain`: Check the validity of the entire chain

`checkBlockHash` function:

In [ ]:
def checkBlockHash(block):
    expectedHash = hashMe( block['contents'] )
    # Raise an error if the hash does not match the block contents
    if block['hash']!=expectedHash:
        raise Exception('Hash does not match contents of block %s'%
                        block['contents']['blockNumber'])
    return

`checkBlockValidity` function:

In [ ]:
def checkBlockValidity(block,parent,state):    
    # We want to check the following conditions:
    # - Each of the transactions are valid updates to the system state
    # - Block hash is valid for the block contents
    # - Block number increments the parent block number by 1
    # - Accurately references the parent block's hash
    parentNumber = parent['contents']['blockNumber']
    parentHash   = parent['hash']
    blockNumber  = block['contents']['blockNumber']
    
    # Check transaction validity; throw an error if an invalid transaction was found.
    for txn in block['contents']['txns']:
        if isValidTxn(txn,state):
            state = updateState(txn,state)
        else:
            raise Exception('Invalid transaction in block %s: %s'%(blockNumber,txn))

    checkBlockHash(block) # Check hash integrity; raises error if inaccurate

    if blockNumber!=(parentNumber+1):
        raise Exception('Hash does not match contents of block %s'%blockNumber)

    if block['contents']['parentHash'] != parentHash:
        raise Exception('Parent hash not accurate at block %s'%blockNumber)
    
    return state

`checkChain` function:

In [ ]:
def checkChain(chain):
    # Work through the chain from the genesis block (which gets special treatment), 
    #  checking that all transactions are internally valid,
    #    that the transactions do not cause an overdraft,
    #    and that the blocks are linked by their hashes.
    # This returns the state as a dictionary of accounts and balances,
    #   or returns False if an error was detected

    
    ## Data input processing: Make sure that our chain is a list of dicts
    if type(chain)==str:
        try:
            chain = json.loads(chain)
            assert( type(chain)==list)
        except:  # This is a catch-all, admittedly crude
            return False
    elif type(chain)!=list:
        return False
    
    state = {}
    ## Prime the pump by checking the genesis block
    # We want to check the following conditions:
    # - Each of the transactions are valid updates to the system state
    # - Block hash is valid for the block contents

    for txn in chain[0]['contents']['txns']:
        state = updateState(txn,state)
    checkBlockHash(chain[0])
    parent = chain[0]
    
    ## Checking subsequent blocks: These additionally need to check
    #    - the reference to the parent block's hash
    #    - the validity of the block number
    for block in chain[1:]:
        state = checkBlockValidity(block,parent,state)
        parent = block
        
    return state

We can now check the validity of the state:

In [ ]:
checkChain(chain)

And even if we are loading the chain from a text file, e.g. from backup or loading it for the first time, we can check the integrity of the chain and create the current state:

In [ ]:
chainAsText = json.dumps(chain,sort_keys=True)
checkChain(chainAsText)

## The final Blockchain Architecture
In a real blockchain network,new nodes would download a copy of the blockchain and verify it (as we just did above), then announce their presence on the peer-to-peer network and start listening for transactions. Bundling transactions into a block, they then pass their proposed block on to other nodes.

We’ve seen how to verify a copy of the blockchain, and how to bundle transactions into a block. If we recieve a block from somewhere else, verifying it and adding it to our blockchain is easy.

Let’s say that the following code runs on Node A, which mines the block:

In [ ]:
import copy
nodeBchain = copy.copy(chain)
nodeBtxns  = [makeTransaction() for i in range(5)]
newBlock   = makeBlock(nodeBtxns,nodeBchain)

Now assume that the newBlock is transmitted to our node, and we want to check it and update our state if it is a valid block:

In [ ]:
print("Blockchain on Node A is currently %s blocks long"%len(chain))

try:
    print("New Block Received; checking validity...")
    state = checkBlockValidity(newBlock,chain[-1],state) # Update the state- this will throw an error if the block is invalid!
    chain.append(newBlock)
except:
    print("Invalid block; ignoring and waiting for the next block...")

print("Blockchain on Node A is now %s blocks long"%len(chain))

## The END
We’ve created all the basic architecture for a blockchain, from a set of state transition rules to a method for creating blocks, to mechanisms for checking the validity of transactions, blocks, and the full chain. We can derive the system state from a downloaded copy of the blockchain, validate new blocks that we receive from the network, and create our own blocks.

The system state that we’ve created is effectively a distributed ledger or database- the core of many blockchains. We could extend this to include special transaction types or full smart contracts.

We haven't done to the networks architecture, public key encryption/decryption, user privacy, and verification steps.